In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
data1 = pd.read_csv('ger_data.csv')
labels = data1.iloc[:,-1]   # Enregistrer la varialbe target
data1 = data1.iloc[:,0:-1]; # Enleveer la colonne target
data1 = data1.drop(columns="Tel") # Enlever la colonne tel


In [2]:
def KMENAS(data):
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(data);
    val = pd.crosstab(labels,kmeans.labels_).values
    #print(val[0].argmax()) Colonne du cluster dominant selon la target good
    bcpg = val[0][1-val[0].argmax()]/(val[0][1-val[0].argmax()]+val[0][val[0].argmax()])
    gcpb = val[1][val[0].argmax()]/(val[1][1-val[0].argmax()]+val[1][val[0].argmax()])
    print("Bad Clients predicted Good : {1:2.2f}% \nGood Clients predicted Bad : {0:2.2f}% ".format(bcpg,gcpb))
    return pd.crosstab(labels,kmeans.labels_)

In [ ]:
def DBSc

#### OPERATIONS

# Name not found step

In [3]:
data_cat = data1.select_dtypes(include=['object'])
data_cat_og = data1.select_dtypes(include=['object'])
data_num_og = data1._get_numeric_data()
data_num = data1._get_numeric_data()

for col in data_cat:
    onehot = pd.get_dummies(data_cat[col])
    data_cat = data_cat.drop(col,axis = 1)
    data_cat = data_cat.join(onehot)
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_num_sc = pd.DataFrame(scaler.fit_transform(data_num), columns=data_num.columns)

data = [data_num_sc,data_cat]
data_join = pd.concat(data,axis=1)

In [4]:
datalist = ["data_num_sc","data_cat","data_join"]

# MODELS

### Quanti

In [5]:
KMENAS(data_num_sc)

Bad Clients predicted Good : 0.85% 
Good Clients predicted Bad : 0.16% 


col_0,0,1
Actual_Score,,
1,109,591
2,46,254


In [6]:
reduced_data = PCA(n_components=2).fit_transform(data_num_sc)
reduced_num = pd.DataFrame(reduced_data)
KMENAS(reduced_num)

Bad Clients predicted Good : 0.57% 
Good Clients predicted Bad : 0.47% 


col_0,0,1
Actual_Score,,
1,328,372
2,128,172


### Quali

In [16]:
KMENAS(data_cat)

Bad Clients predicted Good : 0.49% 
Good Clients predicted Bad : 0.42% 


col_0,0,1
Actual_Score,,
1,294,406
2,153,147


In [17]:
reduced_data = PCA(n_components=2).fit_transform(data_cat)
reduced_cat = pd.DataFrame(reduced_data)
KMENAS(reduced_cat)

Bad Clients predicted Good : 0.43% 
Good Clients predicted Bad : 0.45% 


col_0,0,1
Actual_Score,,
1,317,383
2,172,128


### Both

In [18]:
KMENAS(data_join)

Bad Clients predicted Good : 0.49% 
Good Clients predicted Bad : 0.42% 


col_0,0,1
Actual_Score,,
1,406,294
2,147,153


In [19]:
reduced_data = PCA(n_components=2).fit_transform(data_join)
reduced_join = pd.DataFrame(reduced_data)
KMENAS(reduced_join)

Bad Clients predicted Good : 0.43% 
Good Clients predicted Bad : 0.45% 


col_0,0,1
Actual_Score,,
1,385,315
2,130,170
